<a href="https://colab.research.google.com/github/megane-chan/RED_dialogue/blob/main/dialogue_v1_milan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
![ -e RED_dialogue ] && rm -r RED_dialogue
!git clone https://github.com/megane-chan/RED_dialogue
!cd RED_dialogue

Cloning into 'RED_dialogue'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), 10.58 KiB | 677.00 KiB/s, done.


In [ ]:
!pip install -q transformers datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

ds = load_dataset("daily_dialog", "train")


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
label_names =  ["dummy", "inform", "question", "directive", "commissive"]#, "no emotion", "anger", "disgust", "fear", "happiness"]
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}


In [ ]:
def dataprep(train):
  #train['dialog'] =  [item for sublist in train['dialog'] for item in sublist]

  encoding = tokenizer.batch_encode_plus(train['dialog'], padding="max_length", truncation=True, max_length=128)
  all_labels = []


  for j in range(0, len(encoding["input_ids"])-1):
    labels = np.zeros((128,len(label_names)))
    labels[:, train['act'][j]] = 1
    
    #labels[:, train['emotion'][j] + 5 ] = 1
    all_labels = all_labels + [labels.tolist()]

  encoding['labels'] = all_labels
  return encoding

Idea: introduce a learned dialog embedding which is learned, but it allows you to match the similarity of different dialog interations. Calculated similar to sentance enmbedding, using a self-attention head to learn dialog embeddings  using \[MASK] token to hide sentances and guessing which sentances are hidden. Essentially, what BERT does to sentances on a dialog level


I googled a little and it already exists!!!! Yay.


https://arxiv.org/abs/2012.01775 - DialogBERT: uses the dialog embedding that I was thinking about. Also uses a very similar learning methodology to what I was thinking about.

https://aclanthology.org/N19-1373/ - Similar but uses bidirectional RNN's instead of bert encoder model. Does use self-attention

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

In [ ]:
encoded = ds.map(dataprep)

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

IndexError: ignored

In [ ]:
#encoded['train'][0]

In [ ]:
encoded.set_format("torch")

In [ ]:
encoded

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
batch_size = 8
metric_name = "f1"
#from transformers import LongformerForSequenceClassification
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="single_label_classification", 
                                                           num_labels=len(label_names),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
encoded['train'][0]['labels'].type()


'torch.FloatTensor'

In [ ]:
encoded['train']['input_ids'][0]

tensor([[ 101, 2360, 1010,  ...,    0,    0,    0],
        [ 101, 2017, 2113,  ...,    0,    0,    0],
        [ 101, 2054, 2079,  ...,    0,    0,    0],
        ...,
        [ 101, 4165, 2307,  ...,    0,    0,    0],
        [ 101, 2204, 1012,  ...,    0,    0,    0],
        [ 101, 2035, 2157,  ...,    0,    0,    0]])

In [ ]:
#forward pass
#outputs = model(input_ids=encoded['train']['input_ids'][0][0].unsqueeze(0), labels=encoded['train'][0]['labels'].unsqueeze(0))
#outputs

#encoded['train'][0]['labels'].unsqueeze(0)
#encoded['train']['input_ids'][0].unsqueeze(0)

tensor([[[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]]])